In [ ]:
import sys
sys.path.insert(0, r'D:\Github\zi_labrad')

%matplotlib inline
import os
from time import sleep
from importlib import reload
import logging
import matplotlib.pyplot as plt
import numpy as np
import qcodes as qc


logging.basicConfig(format='%(asctime)s | %(name)s [%(levelname)s] : %(message)s',
                    level=logging.INFO
                    )

In [ ]:
import labrad
from labrad.units import Unit,Value
_unitSpace = ('V','mV','us','ns','s','GHz','MHz','kHz','Hz','dBm','rad','None')
V, mV, us, ns,s, GHz, MHz,kHz,Hz, dBm, rad,_l = [Unit(s) for s in _unitSpace]

from zilabrad import *
from zilabrad.pyle.util import sweeptools
ar = sweeptools.RangeCreator()

In [ ]:
_default_modes = [1,2,3,4,5]
devices = [None]*len(_default_modes)

cxn=labrad.connect()
ss = switchSession(cxn,user='hwh',session=None)

In [ ]:
Registry = qc.config
Reg = Registry.update_config('M:\Registry\qcodesrc.json')
# Reg now is only a wrapped dict

reg = Reg['user']
# reg give the parameters that we set

In [ ]:
def checkID():
    if 'devicesID' not in reg.keys():
        devicesid = {
         'microwave_source_readout': '192.168.1.241',
         'microwave_source_xy': '192.168.1.240',
         'yokogawa_dc_ip': '192.168.1.239',
         'zi_hd_id': 'dev8334',
         'zi_qa_id': 'dev2591'}
        Registry.add('devicesID',devicesid)
        Registry.save_to_home()
    else:
        devicesid = reg['devicesID']
        
    logging.info(devicesid)

In [ ]:
checkID()
reg['devicesID']

In [ ]:
def bringup_device(modes,reg):
    zH = zurichHelper
    dev = reg['devicesID']
    for m in modes: 
        if m == 1:
            qa = zH.zurich_qa(dev['zi_qa_id'])
            devices[m-1] = qa
        if m == 2:
            hd = zH.zurich_hd(dev['zi_hd_id'])
            devices[m-1] = hd
        if m == 3:
            mw = zH.microwave_source(dev['microwave_source_xy'],'mw')
            devices[m-1] = mw
        if m == 4:
            mw_r = zH.microwave_source(dev['microwave_source_readout'],'mw_r')
            devices[m-1] = mw_r
        if m == 5:
            wfs = waveforms.waveform()
            devices[m-1] = wfs
    return 

In [ ]:
# bringup all
bringup_device(_default_modes,reg)

In [ ]:
# bringup_device([3],reg)

In [ ]:
reload(mp)
mp.exp_devices = devices
mp.exp_devices

In [ ]:
# mp.s21_scan(ss,freq=ar[6.0:6.1:0.01,GHz])